---
# General Formulation of MPEC for economic models
---

Assume an economic model is described by the parameter vector $\theta$ and a state vector $x$ as well as some endogeneous variables $\sigma$. The data consists of $X = \{x_i, d_i\}^M_{i=1}$. $x_i$ is the observed state and $d_i$ is the observed equilibrium outcome of the underlying decision model. $M$ is the number of data points.

Generally $\sigma$ depends on the parameters $\theta$ through a set of equlibrium conditions. This also includes Bellman Equations. This consistency is expressed by the follwoing condition: 

\begin{align}
h(\theta, \sigma) = 0.
\end{align}

For a given $\theta$, let $\Sigma(\theta)$ denote the set of $\sigma(\theta)$ for which the equilibrium conditions hold, i.e. $h(\theta,\sigma)=0$. 

\begin{align}
\Sigma(\theta) := {\sigma:h(\theta, \sigma)=0}.
\end{align}

Let $\sigma\hat(\theta)$ denote an element of the above set. In the case of the a dynamic discrete-choice model, this represents the expected value function. This set does not have to be a singleton for a given $\theta$.

For the case of multiple $\sigma\hat(\theta)$ the log likehood function is:

\begin{align}
\hat\theta = argmax_\theta \{max_{\hat\sigma(\theta)\in\Sigma(\theta)} L(\theta,\hat\sigma(\theta); X)\}.
\end{align}

**NFXP**

The nested fixed point algorithm now attempts to evaluate the above log likelihood function by searching for $\theta$ that maximizes it in an outer loop and then, for a given $\theta$, caluclates all $\hat\sigma(\theta)\in\Sigma(\theta)$ while choosing the one that gives the highest log likelihood (in case $\Sigma(\theta)$ is not a singleton).

**MPEC** 

MPEC now goes a different way of ensuring that a guess of $\theta$ is consistent with the equilibrium condition $h(\theta, \sigma)=0$. It models $\sigma$ explicitly as another parameter vector and makes sure that the equilibrium condition holds by imposing it as a constraint to the new log likelihood function. More formally the log likehood function now becomes $L(\theta, \sigma; X)$ and the optimization problem now becomes a constrained optimization looking like the following:

\begin{align}
max_{(\theta, \sigma)} L(\theta, \sigma; X) 
\end{align}
\begin{align}
subject\: to\: h(\theta, \sigma) = 0.
\end{align}

Su & Judd (2012) provide a proof that those two formulations are actually equivalent in the sense that they yield the same solutions for $\hat\theta$. Further they state that MPEC also works for GMM (see Dubé, Fox and Su (2012)) as well as simulated maximum likelihood.

---
# Practical Consequences following Rust's Engine Replacement Problem
---

In the case of Rust's Engine Replacement problem where we discretize the mileage into a grid of 90 (i.e. $\sigma$ consists of 90 elements) and assume the cost function to be linear (i.e. $\theta$ consists of two elements), the MPEC problem consists of 90 nonlinear constraints and 92 parameters. Depending on the complexity of the model the amount of non-linear constraints will quickly become quite large. Su & Judd (2012) report, though, that they successfully solved a structural model with 100,042 constrains and 100,052 parameters within an hour on a 12 GB RAM work station. The constraint Jacobian as well as the Hessian of the Lagrangian were sparse, though, and they provided analytical first and second order derivatives as well as their sparsity patterns. This was possible by using KNITRO and won't be with nlopt (maybe with ipopt, though?).

The big advantage of MPEC is that it is comparably easy to code up as the whole code regarding the solving of the economic model is not needed. Furthermore it is not necessary to calculate contraction mappings and so on for each guess of $\theta$ which can make MPEC faster if the underlying model is quite complex (has many dimensions) and therefore takes for example a lot of contraction steps to solve the fixed point. This only holds, though, as long as the constraint Jacobian as well as the Hessian of the Lagrangian are sparse (see Dubé, Fox, Su (2012)).

---
# Applicability to respy
---

Please read this section as the starting point for discussion.

- Theoretically it seems possible to use MPEC for the estimation. In that case it seems necessary to impose as many nonlinear conditions as there are possible $EMAX$. All those are also modelled explicitly in the log likelihood function as parameters to be estimated. Hence, their dependence on the parameter vector $\theta$ does not enter the log likelihood function. In the language from before $EMAX$ would be our $\sigma$ and the condition that $\sigma$ and $\theta$ are consistent with each other is cast to the constraints.

- In the case of ruspy, this seriously simplified the calculation as the whole calculation for the fixed point (contractin mapping and N-K iterations) was omitted. So the increase in dimensionality of the problem for MPEC was offset by the fact that all the fixed point iterations were not needed anymore. As far as I can see it (although I cannot claim to have fully grasped what is going on in respy as well as Keane & Wolpin (1994, 1997)), this advantage won't be as prominent for respy as this problem has a finite horizon and is solved by backward induction. Hence, no fixed point has to be calculated and therefore no iteration is necessary. With MPEC, though, the backward induction does not have to be conducted all the way through. Instead the constraints would be such that they are set up so that the backward induction holds for every guess of $(\theta, EMAX)$.

- Not sure whether that is advantageous enough (or even practically feasible...). Also, I don't know yet how to incorporate the interpolation into MPEC. 

### References


Su, C.‐L. and Judd, K.L. (2012), Constrained Optimization Approaches to Estimation of Structural Models. Econometrica, 80: 2213-2230. doi:10.3982/ECTA7925

Dubé, J.‐P., Fox, J.T. and Su, C.‐L. (2012), Improving the Numerical Performance of Static and Dynamic Aggregate Discrete Choice Random Coefficients Demand Estimation. Econometrica, 80: 2231-2267. doi:10.3982/ECTA8585

Dong, Baiyu and Hsieh, Yu-Wei and Zhang, Xing, Implementing Maximum Likelihood Estimation of Empirical Matching Models: MPEC versus NFXP (April 20, 2017). USC-INET Research Paper No. 17-16. Available at SSRN: https://ssrn.com/abstract=2959116 or http://dx.doi.org/10.2139/ssrn.2959116